# Хакатон DODO pizza

## В этом блокноте я поделюсь своим решением к этому [хакатону](https://datasecrets.ru/hackathons/7) от DODO pizza по оценки вероятности использования промо акции

In [1]:
# Импортируем нужные модули 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
# Загружаем предоставленные данные
orders = pd.read_csv("data/orders.csv")
train_target = pd.read_csv("data/train_target.csv")
clients_promo_october = pd.read_csv("data/clients_promo_october.csv")
mobile_events = pd.read_csv("data/mobile_events.csv")
test = pd.read_csv("data/test.csv")
submit = pd.read_csv("data/submit.csv")

### Просматриваем данные

In [3]:
orders

,OrderUUId,addressId,deliverySectorId,ClientUUId,Date,SaleDate,UnitUUId,NewClient,ClientOrderNumber,ProductUUId,CategoryId,ProductTotalPrice,MenuPrice,OrderState,OrderPaymentType,OrderTotalPrice,OrderType,apply_promo
0,020DD76CCD949AE011EDE031ED5D76FD,NaN,NaN,000D3AAC9DCABB2E11EBE0614C89D044,2023-04-21,2023-04-21T20:51:26.000Z,000D3A2480C380DA11E692CFB9F92484,0,18,000D3A240C71BE9A11E719BE2AB264A6,4,1.0,45.0,4,2,1.0,3,0
1,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EC9AABAD354BE051CCF1EE1F83A8D0,1,485.0,485.0,4,2,1164.0,1,0
2,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EB846583CB3BDEA80D1A7F2E184D40,1,589.0,589.0,4,2,1164.0,1,0
3,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB267FD,4,45.0,45.0,4,2,1164.0,1,0
4,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB264A6,4,45.0,45.0,4,2,1164.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611734,FEE4A40E62DA936011EE0DA9C66D87C8,0.0,0.0,000D3A39D824A82E11EA2BB7F48841A0,2023-06-18,2023-06-18T15:34:13.000Z,000D3A22FA54A81411E95B9E966DC9B2,0,55,11EA5EE49DCBD9124A66F764872C8E20,1,594.0,719.0,4,2,1509.0,1,0
611735,FEE4A40E62DA936011EE0DA9C66D87C8,0.0,0.0,000D3A39D824A82E11EA2BB7F48841A0,2023-06-18,2023-06-18T15:34:13.000Z,000D3A22FA54A81411E95B9E966DC9B2,0,55,000D3A39D824A82E11E9A18231161F87,1,395.0,479.0,4,2,1509.0,1,0
611736,FEE7759E6F988C7111EDC1BDEEDF6376,9723140.0,4103.0,000D3A22FA54A81411E9BC5EF27EE5BF,2023-03-24,2023-03-24T15:50:51.000Z,000D3A24D2B7A94311E8706B7529EBFD,0,28,000D3A240C71BE9A11E719BE2AB2D427,3,169.0,169.0,4,2,557.0,1,0
611737,FEE7759E6F988C7111EDC1BDEEDF6376,9723140.0,4103.0,000D3A22FA54A81411E9BC5EF27EE5BF,2023-03-24,2023-03-24T15:50:51.000Z,000D3A24D2B7A94311E8706B7529EBFD,0,28,11ED08AA93691DC615EE94EB19CD7070,3,189.0,189.0,4,2,557.0,1,0


In [4]:
clients_promo_october

,ClientUUId,Id,LocalBeginDate,LocalEndDate,OrderType,OrderPrice,Discount
0,000D3A20F23EA95811E7B7104AC23CD3,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,25
1,000D3A20F23EA95811E7B7D79FAABAA1,5,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,300
2,000D3A20F23EA95811E7B8BD71C73877,7,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,200
3,000D3A20F23EA95811E7B99F3ED09FC8,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,30
4,000D3A20F23EA95811E7BD373E79565E,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,20
...,...,...,...,...,...,...,...
477270,82EDCEB57826A0A811ED680C33F09E52,6,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,20
477271,82F78FC7401A9ADD11EDFA15A350D31F,5,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,200
477272,82F78FC7401AA4A811EDF992437D8CC6,5,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,200
477273,82F9662CB356B7E011EE167FC961059B,6,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,20


In [5]:
mobile_events

,ClientUUId,VisitToken,EventName,Platform,Timestamp
0,000D3A22FA54A81611EB315CF5443815,976627AD-76DC-41D9-981E-F2A6CA14B3A9,screen_menu,ios,2023-10-29T16:14:21.343Z
1,2A4A64AF719E994511EE3F2D8B31E188,5b22e610-c71e-4b6a-a842-71e8575ece87,screen_menu,android,2023-10-29T11:04:36.4840000Z
2,000D3AAC977BBB2F11ECDD081F5481AE,BDCEB72F-2B6A-4565-801A-6CB729279C6F,open_app,ios,2023-10-29T11:14:34.723Z
3,000D3A21DA51A81411EAE468856F96F1,A66E2DA6-4E2E-4C37-9FBF-627732623212,add_to_cart,ios,2023-10-29T14:03:08.363Z
4,000D3A23B0DC80D811E67F4D0A9CA345,1BE8139F-A3D3-439F-A51B-5CBD4D3FFF1B,screen_menu,ios,2023-10-29T08:35:40.127Z
...,...,...,...,...,...
3916649,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,open_app,ios,2023-10-14T06:00:21.558Z
3916650,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_profile,android,2023-10-13T12:55:57.7520000Z
3916651,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,close_app,ios,2023-10-14T06:00:21.515Z
3916652,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_menu,android,2023-10-13T12:55:56.3030000Z


In [6]:
train_target

,ClientUUId,Id,OrderType,LocalBeginDate,LocalEndDate,OrderPrice,Discount,apply_promo
0,000D3A20F23EA95811E7B99F3ED09FC8,7,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,200,0
1,000D3A20F23EA95811E7B99F3ED09FC8,6,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,25,0
2,000D3A20F23EA95811E7BD373E79565E,5,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,799,200,0
3,000D3A20F23EA95811E7BD373E79565E,5,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,250,0
4,000D3A20F23EA95811E7BDAE71CD3F33,6,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,30,0
...,...,...,...,...,...,...,...,...
27736,82EDBCD6CE60978F11EDC591DCA75401,5,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,799,200,0
27737,82EDBCD6CE60978F11EDC591DCA75401,5,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,250,0
27738,82F78FC7401A9ADD11EDFA15A350D31F,6,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,799,20,0
27739,8606F5E4CA39BF7211EE517E6E1D90B7,5,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,749,200,0


## Работаем с данными

In [7]:
orders['Date'] = pd.to_datetime(orders['Date'])

In [8]:
# Установим текущую дату (для примера 2023-11-01)
current_date = pd.to_datetime('2023-11-01')

In [9]:
# Функция для создания признаков из дат заказов для одного пользователя
def create_features_for_user(df, current_date):
    features = {}
    ClientUUId = df['ClientUUId'].iloc[0]
    features['ClientUUId'] = ClientUUId

    # Количество заказов за последний месяц
    last_month_orders = df[df['Date'] >= current_date - pd.DateOffset(months=1)]
    features['num_orders_last_month'] = len(last_month_orders)

    # Средний интервал времени между заказами
    df = df.sort_values('Date')
    df['days_since_last_order'] = df['Date'].diff().dt.days
    features['mean_days_between_orders'] = df['days_since_last_order'].mean()

    # Время с момента последнего заказа
    last_order_date = df['Date'].max()
    features['days_since_last_order'] = (current_date - last_order_date).days

    # Время от первого до последнего заказа
    first_order_date = df['Date'].min()
    features['days_between_first_and_last_order'] = (last_order_date - first_order_date).days

    # Количество заказов в рабочие дни
    df['weekday'] = df['Date'].dt.weekday  # День недели (0 - понедельник, 6 - воскресенье)
    workdays_orders = df[df['weekday'].isin([0, 1, 2, 3, 4])]  # Рабочие дни (понедельник - пятница)
    features['num_orders_workdays'] = len(workdays_orders)

    # Время с момента последнего использования промокода
    if df['apply_promo'].sum() == 0:
        features['days_since_last_promo'] = 1000
    else:
        last_promo_date = df[df['apply_promo'] == 1]['Date'].max()
        features['days_since_last_promo'] = (current_date - last_promo_date).days

    # Промокоды за последний месяц
    last_month_promo_orders = last_month_orders[last_month_orders['apply_promo'] == 1]
    features['promo_used_last_month'] = int(len(last_month_promo_orders) > 0)

    return features

In [10]:
# Применение функции к каждому пользователю
user_groups = orders.groupby('ClientUUId')
features_list = [create_features_for_user(group, current_date) for name, group in user_groups]

# Создание DataFrame с признаками
features_df = pd.DataFrame(features_list)

features_df.to_csv("data/date_features.csv", index=False)

In [11]:
dates = pd.read_csv('data/date_features.csv')

In [12]:
mobile = pd.get_dummies(mobile_events, columns=['EventName', 'Platform'], dtype=int, prefix='')

In [13]:
# Группируем по пользователю и считаем сумму по каждому указанному столбцу
mobile = mobile.groupby('ClientUUId').agg(
    count=('VisitToken', 'count'),
    _add_to_cart_sum=('_add_to_cart', 'sum'),
    
    param1=('_apply_personal_offer', 'mean'),
    obm1=('_open_bonusaction', 'mean'),
    
    _apply_personal_offer_sum=('_apply_personal_offer', 'sum'),
    _close_app_sum=('_close_app', 'sum'),
    _create_order_sum=('_create_order', 'sum'),
    _open_app_sum=('_open_app', 'sum'),
    _open_bonusaction_sum=('_open_bonusaction', 'sum'),
    _open_product_card_sum=('_open_product_card', 'sum'),
    _remove_from_cart_sum=('_remove_from_cart', 'sum'),
    _screen_cart_sum=('_screen_cart', 'sum'),
    _screen_menu_sum=('_screen_menu', 'sum'),
    _screen_profile_sum=('_screen_profile', 'sum'),
    _android_sum=('_android', 'sum'),
    _ios_sum=('_ios', 'sum')
).reset_index()

In [14]:
# колонки, которые ухудшают скор для некоторых моделей
xg_cols = []  
cat_cols = ['aaa', 'bbb']

In [15]:
train_target['n_promos'] = train_target.groupby('ClientUUId')['LocalEndDate'].transform('count')
test['n_promos'] = train_target.groupby('ClientUUId')['LocalEndDate'].transform('count')

train_target['avg_OrderPrice'] = train_target.groupby('ClientUUId')['OrderPrice'].transform('mean')
train_target['min_OrderPrice'] = train_target.groupby('ClientUUId')['OrderPrice'].transform('min')
train_target['avg_disc'] = train_target.groupby('ClientUUId')['Discount'].transform('mean')
train_target['max_disc'] = train_target.groupby('ClientUUId')['Discount'].transform('max')


test['avg_OrderPrice'] = test.groupby('ClientUUId')['OrderPrice'].transform('mean')
test['min_OrderPrice'] = test.groupby('ClientUUId')['OrderPrice'].transform('min')
test['avg_disc'] = test.groupby('ClientUUId')['Discount'].transform('mean')
test['max_disc'] = test.groupby('ClientUUId')['Discount'].transform('max')


train_target['use_for_delivery'] = (train_target['OrderType'] == '1,2,3').astype(int)
train_target.drop(['OrderType'], axis=1, inplace=True)
test['use_for_delivery'] = (test['OrderType'] == '1,2,3').astype(int)
test.drop(['OrderType'], axis=1, inplace=True)


train_target['avg_del'] = train_target.groupby('ClientUUId')['use_for_delivery'].transform('mean')
train_target['max_del'] = train_target.groupby('ClientUUId')['use_for_delivery'].transform('max')
test['avg_del'] = test.groupby('ClientUUId')['use_for_delivery'].transform('mean')
test['max_del'] = test.groupby('ClientUUId')['use_for_delivery'].transform('max')


train_target['a'] = train_target.OrderPrice > train_target.min_OrderPrice
train_target['b'] = train_target.Discount < train_target.max_disc
train_target['c'] = train_target.use_for_delivery < train_target.max_del
test['a'] = test.OrderPrice > test.min_OrderPrice
test['b'] = test.Discount < test.max_disc
test['c'] = test.use_for_delivery < test.max_del

In [16]:
orders[['price_with_discont', 'price']] = orders.groupby(['OrderUUId'])[['ProductTotalPrice', 'MenuPrice']].transform(sum)

E:\temp\ipykernel_7176\1176280996.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  orders[['price_with_discont', 'price']] = orders.groupby(['OrderUUId'])[['ProductTotalPrice', 'MenuPrice']].transform(sum)


In [17]:
prices = orders.groupby(['ClientUUId'])['MenuPrice'].agg([
    'mean',
    'std'
]).reset_index()
cats = pd.get_dummies(orders[['ClientUUId', 'CategoryId']], columns=['CategoryId'], dtype=int).groupby(['ClientUUId']).mean()

In [18]:
orders.OrderType = (orders.OrderType == 1).astype(int)
orders.OrderState = (orders.OrderState == 4).astype(int)

In [19]:
orders['in_delivery'] = (orders.deliverySectorId == 0).astype(int)

In [20]:
orders.drop(['ProductTotalPrice', 'MenuPrice', 'CategoryId', 'ProductUUId', 'addressId', 'deliverySectorId'], axis=1, inplace=True)

In [21]:
orders.drop_duplicates(inplace=True)

In [22]:
orders = pd.get_dummies(orders, columns=['OrderPaymentType'], prefix='pay')

orders['discont'] = orders.price - orders.price_with_discont

In [23]:
orders = orders.groupby('ClientUUId').agg(
    avg_discont=('discont', 'mean'),
    sum_discont=('discont', 'sum'),
    aaa=('in_delivery', 'mean'),
    bbb=('in_delivery', 'sum'),
    apply_promo_mean1=('apply_promo', 'mean'),
    
    ClientOrderNumber_max=('ClientOrderNumber', 'max'),
    ClientOrderNumber_min=('ClientOrderNumber', 'min'),
    ClientOrderNumber_avg=('ClientOrderNumber', 'mean'),
    OrderState_mean=('OrderState', 'mean'),
    OrderType_mean=('OrderType', 'mean'),
    pay_0_count=('pay_0', 'sum'),
    pay_1_count=('pay_1', 'sum'),
    pay_2_count=('pay_2', 'sum'),
    OrderTotalPrice_mean=('OrderTotalPrice', 'mean'),
    OrderTotalPrice_std=('OrderTotalPrice', 'std'),
    OrderTotalPrice_sum=('OrderTotalPrice', 'sum'),
    price_mean=('price', 'mean'),
    price_sum=('price', 'sum'),
    apply_promo_mean=('apply_promo', 'mean'),
    apply_promo_sum=('apply_promo', 'sum'),
).reset_index()


In [24]:
orders = pd.merge(pd.merge(orders, cats, on='ClientUUId'), prices, on='ClientUUId')

In [26]:
train_target = train_target.rename(columns={'apply_promo': 'target'})
test = test.rename(columns={'apply_promo': 'target'})
test.head()

,ClientUUId,Id,LocalBeginDate,LocalEndDate,OrderPrice,Discount,n_promos,avg_OrderPrice,min_OrderPrice,avg_disc,max_disc,use_for_delivery,avg_del,max_del,a,b,c
0,000D3A20F23EA95811E7C0A95563344E,7,2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200,2,799.0,799,200.0,200,0,0.0,0,False,False,False
1,000D3A20F23EA95811E7C7892A0CE261,5,2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,699,200,2,699.0,699,200.0,200,0,0.0,0,False,False,False
2,000D3A20F23EA95811E7CD686C396528,6,2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,20,2,799.0,799,20.0,20,0,0.0,0,False,False,False
3,000D3A20F23EA95911E7CEA8C574EDAE,5,2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200,2,1024.0,799,200.0,200,0,0.5,1,False,False,True
4,000D3A20F23EA95911E7D4F05C59C978,7,2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200,2,799.0,799,200.0,200,0,0.0,0,False,False,False


## Предобработка данных перед обучением

In [27]:
df = pd.merge(
    pd.merge(
        pd.merge(
            train, orders, on='ClientUUId', how='left'),
        mobile, on='ClientUUId', how='left'),
    dates.iloc[:, [0, 4, 5, 7]], on='ClientUUId', how='left')
df_test = pd.merge(
    pd.merge(
        pd.merge(
            test, orders, on='ClientUUId', how='left'), 
        mobile, on='ClientUUId', how='left'),
    dates.iloc[:, [0, 4, 5, 7]], on='ClientUUId', how='left')

In [28]:
cat_cols.append('days_between_first_and_last_order')
cat_cols.append('num_orders_workdays')
xg_cols.append('promo_used_last_month')

In [29]:
df = pd.get_dummies(df, columns=['Id'], dtype=int)
df_test = pd.get_dummies(df_test, columns=['Id'], dtype=int)

In [30]:
df.drop(['LocalBeginDate', 'LocalEndDate', 'ClientUUId'], axis=1, inplace=True)
df_test.drop(['LocalBeginDate', 'LocalEndDate', 'ClientUUId'], axis=1, inplace=True)

In [31]:
df = df.fillna(0)
df_test = df_test.fillna(0)

In [32]:
X = df.drop(['target'], axis=1)
y = df['target']


In [33]:
lin_cols = ['days_between_first_and_last_order',
                                         'num_orders_workdays',
                                         'promo_used_last_month']

## Обучаем несколько моделей: Линейная регрессия, XGboost и catboost

In [34]:
n_splits = 6
lin_models = []
test_scores = []
train_scores = []
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=52)

lin_X = X.drop(lin_cols, axis=1)
for train_index, test_index in kf.split(X=lin_X, y=y):
    X_train, X_test = lin_X.iloc[train_index], lin_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    


    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    # Сделаем предсказания на тестовой выборке
    lin_test = lin_reg.predict(X_test)
    lin_train = lin_reg.predict(X_train)
    
    test_score = roc_auc_score(y_train, lin_train)
    train_score = roc_auc_score(y_test, lin_test)

    lin_models.append(lin_reg)
    test_scores.append(test_score)
    train_scores.append(train_score)


print("mean score --", np.mean(test_scores, dtype="float16"), np.std(test_scores).round(4))
print("mean score --", np.mean(train_scores, dtype="float16"), np.std(train_scores).round(4))
print("difference:", np.mean(train_scores, dtype="float16") - np.mean(test_scores, dtype="float16"))


ans = np.array([0]*df_test.shape[0])

    
for n, model in enumerate(lin_models):
    ans = ans + model.predict(df_test.drop(lin_cols, axis=1))
    
ans = pd.DataFrame(ans, columns = ['apply_promo'])
ans.to_csv('result/lin.csv', index=False)

mean score -- 0.7886 0.0035
mean score -- 0.7793 0.0168
difference: -0.00928


In [41]:
n_splits = 5
xg_models2 = []
test_scores = []
train_scores = []
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=52)
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

xg_X = X.drop(xg_cols, axis=1)

for train_index, test_index in kf.split(X=xg_X, y=y):
    X_train, X_test = xg_X.iloc[train_index], xg_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.7, random_state=42)
    

    
    xgb = XGBClassifier(
        eval_metric="auc",
        n_estimators=100000,
        random_state=98, 
        verbosity=0, 
        scale_pos_weight=scale_pos_weight,
        # booster='dart',
        # eta = 0.04, 
        max_depth = 3, 
        gamma = 0.9,
        min_child_weight=13,
        # colsample_bylevel=0.5,
        # colsample_bytree=0.9,
        # colsample_bynode=0.35,
        subsample=0.9,
        reg_lambda=0.5,
        device='cuda',
        reg_alpha=0.5,
        early_stopping_rounds=40
    )
    xgb.fit(
        X_train, y_train, 
        eval_set=[[X_val, y_val]], 
        verbose=0
    ) 
    xg_models2.append([xgb, (0, xgb.best_iteration + 1)])

    xgb_test_preds = xgb.predict_proba(X_test,
                                  iteration_range=(0, xgb.best_iteration + 1))
    xgb_train_preds = xgb.predict_proba(X_train,
                                   iteration_range=(0, xgb.best_iteration + 1))
    
    test_score = roc_auc_score(y_test, xgb_test_preds[:, 1])
    train_score = roc_auc_score(y_train, xgb_train_preds[:, 1])
    test_scores.append(test_score)
    train_scores.append(train_score)

print("mean score --", np.mean(test_scores, dtype="float16"), np.std(test_scores).round(4))
print("mean score --", np.mean(train_scores, dtype="float16"), np.std(train_scores).round(4))
print("difference:", np.mean(train_scores, dtype="float16") - np.mean(test_scores, dtype="float16"))


ans = np.array([0]*df_test.shape[0])

for n, model in enumerate(xg_models2):
    ans = ans + model[0].predict_proba(df_test.drop(xg_cols, axis=1), 
                                       iteration_range=model[1])[:, 1]
    
ans = pd.DataFrame(ans, columns = ['apply_promo'])
ans.to_csv('resul/best.csv', index=False)

mean score -- 0.876 0.0117
mean score -- 0.9775 0.0116
difference: 0.10156


In [42]:
n_splits = 5
xg_models = []
test_scores = []
train_scores = []
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=52)
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

xg_X = X.drop(xg_cols, axis=1)

for train_index, test_index in kf.split(X=xg_X, y=y):
    X_train, X_test = xg_X.iloc[train_index], xg_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.7, random_state=42)
    

    
    xgb = XGBClassifier(
        eval_metric="auc",
        n_estimators=100000,
        random_state=52, 
        verbosity=0, 
        scale_pos_weight=scale_pos_weight,
        # booster='dart',
        # eta = 0.04, 
        max_depth = 3, 
        gamma = 0.9,
        min_child_weight=13,
        # colsample_bylevel=0.5,
        colsample_bytree=0.5,
        # colsample_bynode=0.25,
        subsample=0.9,
        reg_lambda=0.6, #<-------------------------------------
        device='cuda',
        reg_alpha=0.5,
        early_stopping_rounds=40
    )
    xgb.fit(
        X_train, y_train, 
        eval_set=[[X_val, y_val]], 
        verbose=0
    ) 
    xg_models.append([xgb, (0, xgb.best_iteration + 1)])

    xgb_test_preds = xgb.predict_proba(X_test,
                                  iteration_range=(0, xgb.best_iteration + 1))
    xgb_train_preds = xgb.predict_proba(X_train,
                                   iteration_range=(0, xgb.best_iteration + 1))
    
    test_score = roc_auc_score(y_test, xgb_test_preds[:, 1])
    train_score = roc_auc_score(y_train, xgb_train_preds[:, 1])
    test_scores.append(test_score)
    train_scores.append(train_score)

print("mean score --", np.mean(test_scores, dtype="float16"), np.std(test_scores).round(4))
print("mean score --", np.mean(train_scores, dtype="float16"), np.std(train_scores).round(4))
print("difference:", np.mean(train_scores, dtype="float16") - np.mean(test_scores, dtype="float16"))


ans = np.array([0]*df_test.shape[0])

for n, model in enumerate(xg_models):
    ans = ans + model[0].predict_proba(df_test.drop(xg_cols, axis=1), 
                                       iteration_range=model[1])[:, 1]
    
ans = pd.DataFrame(ans, columns = ['apply_promo'])
ans.to_csv('result/xg.csv', index=False)

mean score -- 0.882 0.0134
mean score -- 0.993 0.0066
difference: 0.1113


In [43]:
n_splits = 5
cat_models = []
test_scores = []
train_scores = []
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=52)
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

cat_X = X.drop(cat_cols, axis=1)


for train_index, test_index in kf.split(X=cat_X, y=y):

    
    X_train, X_test = cat_X.iloc[train_index], cat_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.7, random_state=42)
    

    
    
    clf = CatBoostClassifier(iterations=2000,
                             thread_count=-1,
                             random_seed=67,
                             eta = 0.3,
                             eval_metric='AUC',
                             l2_leaf_reg=5,
                             min_data_in_leaf=15,
                             colsample_bylevel=0.5,
                             max_bin=20,
                             scale_pos_weight=scale_pos_weight,
                             subsample =0.8,
                             depth = 3
                            )

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0,
        use_best_model=True,
        early_stopping_rounds=50,
    )

    cat_models.append(clf)
    clf_test_preds = clf.predict_proba(X_test)
    clf_train_preds = clf.predict_proba(X_train)
    
    test_score = roc_auc_score(y_test, [i[1] for i in clf_test_preds])
    train_score = roc_auc_score(y_train, [i[1] for i in clf_train_preds])
    test_scores.append(test_score)
    train_scores.append(train_score)
    # print('test score', test_score)
    # print('train score', train_score)
    # print()

print("mean score --", np.mean(test_scores, dtype="float16"), np.std(test_scores).round(4))
print("mean score --", np.mean(train_scores, dtype="float16"), np.std(train_scores).round(4))
print("difference:", np.mean(train_scores, dtype="float16") - np.mean(test_scores, dtype="float16"))


ans = np.array([0]*df_test.shape[0])


for n, model in enumerate(cat_models):
    ans = ans + model.predict_proba(df_test.drop(cat_cols, axis=1))[:, 1]
    
ans = pd.DataFrame(ans, columns = ['apply_promo'])
ans.to_csv('result/cat.csv', index=False)

mean score -- 0.8604 0.0215
mean score -- 0.971 0.0228
difference: 0.11084


## По метрикам видно, что лучший результат показал XGboost - 0.74